In [1]:
#pip install tensorflow-datasets
#github repo: https://github.com/tensorflow/datasets

import tensorflow_datasets as tfds
import tensorflow as tf

In [2]:
from random import seed
from random import random

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

Below code will create a network with 2 input neurons, 1 hidden neuron and 2 output neurons.

- weights for hidden neuron will be coming from 2 input neurons + 1 bias
- weights for output neuron will be coming from 1 hidden neuron + 1 bias

In [3]:
seed(42)
network = initialize_network(2, 1, 2)
for layer in network:
	print(layer)

[{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926]}]
[{'weights': [0.22321073814882275, 0.7364712141640124]}, {'weights': [0.6766994874229113, 0.8921795677048454]}]


### Forward Propagate:
- Neuron Activation
- Neuron Transfer
- Forward Propagation

In [4]:
#Neuron activation function
#activation = sum(weight_i * input_i) + bias

def activate_neuron(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return activation

In [5]:
#Neuron transfer activation
#using sigmoid function

from math import exp

def transfer(activation):
    return 1.0/(1.0 + exp(-activation))

In [6]:
#Forward propagate
#This will return the outputs from the last layer

def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate_neuron(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [7]:
#Test forward propagation

row = [1, 0]
output = forward_propagate(network, row)
print(output)

[0.710090547112879, 0.7982323416770263]


### Backpropagate error
- Transfer Derivative
- Error Backpropagation

In [8]:
#Transfer derivative - slope of the output value of a neuron
#for sigmoid transfer function - derivative = output * (1.0 - output)

def transfer_derivative(output):
    return output * (1.0 - output)

In [9]:
#Error backpropagation
#error for output layers = (expected - output) * transfer_derivative(output) 
#error for hidden layers = (weight_k * error_j) * transfer_derivative(output)
#error_j is the error signal from the jth neuron in the output layer, 
#weight_k is the weight that connects the kth neuron to the current neuron and 
#output is the output for the current neuron.

def backpropagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

In [10]:
#test backpropagation

expected = [0, 1]
backpropagate_error(network, expected)
for layer in network:
	print(layer)

[{'weights': [0.6394267984578837, 0.025010755222666936, 0.27502931836911926], 'output': 0.7139111559058509, 'delta': -0.002172917106539774}]
[{'weights': [0.22321073814882275, 0.7364712141640124], 'output': 0.710090547112879, 'delta': -0.14618063323611788}, {'weights': [0.6766994874229113, 0.8921795677048454], 'output': 0.7982323416770263, 'delta': 0.032496188653557974}]


### Train Network
- Update weights
- Train network

In [22]:
#Update weights
#weight = weight + learning_rate*input*error

def update_weights(network, row, learning_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i!=0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += learning_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += learning_rate * neuron['delta']

In [23]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		sum_error = 0
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
			backpropagate_error(network, expected)
			update_weights(network, row, l_rate)
		print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [24]:
# Test training backprop algorithm
seed(1)
dataset = [[2.7810836,2.550537003,0],
	[1.465489372,2.362125076,0],
	[3.396561688,4.400293529,0],
	[1.38807019,1.850220317,0],
	[3.06407232,3.005305973,0],
	[7.627531214,2.759262235,1],
	[5.332441248,2.088626775,1],
	[6.922596716,1.77106367,1],
	[8.675418651,-0.242068655,1],
	[7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
	print(layer)

>epoch=0, lrate=0.500, error=6.350
>epoch=1, lrate=0.500, error=5.531
>epoch=2, lrate=0.500, error=5.221
>epoch=3, lrate=0.500, error=4.951
>epoch=4, lrate=0.500, error=4.519
>epoch=5, lrate=0.500, error=4.173
>epoch=6, lrate=0.500, error=3.835
>epoch=7, lrate=0.500, error=3.506
>epoch=8, lrate=0.500, error=3.192
>epoch=9, lrate=0.500, error=2.898
>epoch=10, lrate=0.500, error=2.626
>epoch=11, lrate=0.500, error=2.377
>epoch=12, lrate=0.500, error=2.153
>epoch=13, lrate=0.500, error=1.953
>epoch=14, lrate=0.500, error=1.774
>epoch=15, lrate=0.500, error=1.614
>epoch=16, lrate=0.500, error=1.472
>epoch=17, lrate=0.500, error=1.346
>epoch=18, lrate=0.500, error=1.233
>epoch=19, lrate=0.500, error=1.132
[{'weights': [-1.4688375095432327, 1.850887325439514, 1.0858178629550297], 'output': 0.029980305604426185, 'delta': -0.0059546604162323625}, {'weights': [0.37711098142462157, -0.0625909894552989, 0.2765123702642716], 'output': 0.9456229000211323, 'delta': 0.0026279652850863837}]
[{'weights

In [25]:
# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

In [26]:
for row in dataset:
	prediction = predict(network, row)
	print('Expected=%d, Got=%d' % (row[-1], prediction))

Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
